# Imports and load data

In [1]:
import os
from langchain_community.document_loaders import DirectoryLoader
from dotenv import load_dotenv

load_dotenv(override=True)

model_name = "gpt-4o-mini"

# RAG

In [2]:
""" Setup RAG; embeddings and chain"""

from rag_utils import setup_rag_embeddings, build_rag_chain

retriever = setup_rag_embeddings(data_dir="input/", chroma_path="chroma")
rag_chain = build_rag_chain(retriever=retriever, model_name=model_name)

amount of documents used:  3
Saved 309 chunks to chroma.


e:\Repositories\graphrag-businessqa-evaluation\env\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Map generated questions and ground_truths into evaluation dataset


In [15]:
import json
from datasets import Dataset

with open('synthetic_data_big_context.json', 'r') as f:
    synthetic_data = json.load(f)

if isinstance(synthetic_data, dict) and 'responses' in synthetic_data:
    synthetic_data = synthetic_data['responses']

queries = [item.get('question', "") for item in synthetic_data]
ground_truths = [item.get('ground_truth', "") for item in synthetic_data]
contexts = [item.get('context', "") for item in synthetic_data]

answers = []
retrieved_contexts = []

for query in queries:
    answer = rag_chain.invoke(query)
    answers.append(answer)
    print("Query: ",query)
    print("Anwer: ",answer)
    retrieved_context = [doc.page_content for doc in retriever.invoke(query)]
    retrieved_contexts.append(retrieved_context)
    print("Retrieved context:",retrieved_context)


Query:  How can SAP leverage its extensive data assets and AI capabilities to enhance customer experience while simultaneously driving operational efficiency across its product portfolio?
Anwer:  SAP can enhance customer experience and drive operational efficiency by infusing Business AI across its product portfolio, ensuring AI is relevant, reliable, and responsible. By leveraging extensive data assets from over 25,000 customer tenants, SAP can train AI models that are tailored to specific business processes, delivering real-time insights. This integration of AI into core business functions allows for improved decision-making and streamlined operations, ultimately benefiting customers.
Retrieved context: ['SAP is uniquely positioned to help our customers benefit from artificial intelligence by delivering Business AI, which we define as:\n- **Relevant:** AI delivered, from day one, across all SAP applications that customers use to run their businesses.\n- **Reliable:** AI based on uniq

In [103]:
# populate eval dataset
from ragas import SingleTurnSample, EvaluationDataset

evaluation_samples = []

for query, answer, retrieved_context, ground_truth in zip(queries, answers, retrieved_contexts, ground_truths):
    sample = SingleTurnSample(
        user_input=query,
        response=answer,
        reference=ground_truth,
        retrieved_contexts=retrieved_context
    )
    evaluation_samples.append(sample)

evaluation_dataset = EvaluationDataset(samples=evaluation_samples)



In [68]:
print(evaluation_dataset.samples)
print(type(evaluation_dataset.samples))
print(type(evaluation_dataset.samples[0]))
print(dir(evaluation_dataset.samples[0]))



[SingleTurnSample(user_input='How can SAP leverage its extensive data assets and AI capabilities to enhance customer experience while simultaneously driving operational efficiency across its product portfolio?', retrieved_contexts=['SAP is uniquely positioned to help our customers benefit from artificial intelligence by delivering Business AI, which we define as:\n- **Relevant:** AI delivered, from day one, across all SAP applications that customers use to run their businesses.\n- **Reliable:** AI based on unique business data and business process context to ensure truthful, real-time results, all following the same enterprise standards that customers expect from SAP.\n- **Responsible:** AI built on leading ethics and data privacy standards.\n\nSAP will be the market leader for Business AI by 2025. To get there we must:\n- **Leverage our unique data assets:** SAP cloud customers have entrusted us with access to data from over 25,000 customer tenants to build new products, including tra

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas.metrics import Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model=model_name))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())


metrics = [
    FactualCorrectness(llm=evaluator_llm, mode="precision", name="FactualCorrectness_Precision"),
    FactualCorrectness(llm=evaluator_llm, mode="recall", name="FactualCorrectness_Recall"),
    FactualCorrectness(llm=evaluator_llm, mode="f1", name="FactualCorrectness_F1"),
    Faithfulness(llm=evaluator_llm), 
 
]
results = evaluate(dataset=evaluation_dataset, metrics=metrics)
df = results.to_pandas()

Evaluating:  25%|██▌       | 14/56 [02:55<05:10,  7.39s/it]Exception raised in Job[12]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Evaluating:  27%|██▋       | 15/56 [03:00<04:28,  6.55s/it]Exception raised in Job[13]: TimeoutError()
Exception raised in Job[4]: TimeoutError()
Exception raised in Job[14]: TimeoutError()
Exception raised in Job[2]: TimeoutError()
Exception raised in Job[1]: TimeoutError()
Exception raised in Job[9]: TimeoutError()
Exception raised in Job[16]: TimeoutError()
Evaluating:  84%|████████▍ | 47/56 [06:00<00:56,  6.33s/it]Exception raised in Job[33]: TimeoutError()
Exception raised in Job[32]: TimeoutError()
Exception raised in Job[34]: TimeoutError()
Exception raised in Job[38]: TimeoutError()
Evaluating: 100%|██████████| 56/56 [07:27<00:00,  7.99s/it]


# Results

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 250) # default is 50 / None for unlimited

df

# context recall = measures how many of the relevant documents (or pieces of information) were successfully retrieved
# factual correctness (precision) = proportion of correct claims made that also found in the reference / how many are correct 
# factual correctness (recall) = proportion of facts in the reference that are also present in the response / how many are found
# faithfulness = are the claims made in the response supported by the retrieved context
# semantic similarity = how similar the response is to the ground truth
 


,user_input,retrieved_contexts,response,reference,FactualCorrectness_Precision,FactualCorrectness_Recall,FactualCorrectness_F1,faithfulness
0,How can SAP leverage its extensive data assets and AI capabilities to enhance customer experience while simultaneously driving operational efficiency across its product portfolio?,[],SAP has a significant opportunity to leverage its extensive data assets and AI capabilities to enhance customer experience and drive operational efficiency. The integration of these technologies can lead to a more personalized and responsive serv...,"SAP can leverage its extensive data assets by embedding AI capabilities into its customer relationship management (CRM) and enterprise resource planning (ERP) systems. By utilizing predictive analytics, SAP can provide personalized customer exper...",0.52,NaN,NaN,0.0
1,What strategies can SAP implement to ensure that its transition to a cloud-first model not only meets customer demands but also enhances its sustainability initiatives?,[],"To ensure that SAP's transition to a cloud-first model effectively meets customer demands while enhancing its sustainability initiatives, several key strategies may be implemented. These strategies focus on leveraging technology, fostering partne...","SAP can implement a multi-faceted strategy that includes optimizing its cloud infrastructure for energy efficiency, utilizing renewable energy sources, and developing sustainability-focused applications. By adopting a green cloud strategy, SAP ca...",NaN,0.69,NaN,0.0
2,How can SAP utilize generative AI to enhance its product development processes while ensuring compliance with industry regulations?,[],"## Enhancing Product Development with Generative AI\n\nSAP has the opportunity to significantly enhance its product development processes through the integration of generative AI. This technology can automate design testing and innovation, leadin...","SAP can utilize generative AI to streamline product development by automating the generation of design specifications, testing scenarios, and compliance documentation. By integrating AI tools that can analyze regulatory requirements and generate ...",NaN,NaN,0.40,1.0
3,In what ways can SAP's Business Technology Platform (BTP) be leveraged to foster collaboration across its diverse product lines while enhancing customer engagement?,[],# Leveraging SAP's Business Technology Platform (BTP) for Collaboration and Customer Engagement\n\nSAP's Business Technology Platform (BTP) serves as a pivotal tool for organizations aiming to enhance collaboration across its diverse product line...,"SAP's Business Technology Platform (BTP) can be leveraged to create a unified ecosystem that integrates various product lines, enabling seamless data sharing and collaboration. By utilizing BTP's capabilities for application development, integrat...",NaN,NaN,NaN,1.0
4,How can SAP align its AI initiatives with its corporate sustainability goals to create a competitive advantage in the market?,[],"## Aligning AI Initiatives with Corporate Sustainability Goals\n\nSAP has a significant opportunity to align its AI initiatives with its corporate sustainability goals, which may create a competitive advantage in the market. This alignment can be...","SAP can align its AI initiatives with corporate sustainability goals by developing AI solutions that specifically address environmental challenges, such as optimizing resource usage and reducing waste in supply chains. By leveraging AI to analyze...",NaN,NaN,0.60,1.0
5,"What role does SAP's partner ecosystem play in enhancing the capabilities of its cloud solutions, particularly in the context of AI and sustainability?",[],"## Role of SAP's Partner Ecosystem in Cloud Solutions\n\nSAP's partner ecosystem plays a pivotal role in enhancing the capabilities of its cloud solutions, particularly through the integration of advanced AI technologies and the promotion of sust...",SAP's partner ecosystem plays a crucial role in enha

# GraphRAG

## Indexing

In [ ]:
# graphrag initial setup
!mkdir -p ./graphrag
!python -m graphrag init --root ./graphrag

The syntax of the command is incorrect.




⠋ GraphRAG Indexer 
Initializing project at E:\Repositories\graphrag-businessqa-evaluation\graphrag
⠋ GraphRAG Indexer 


┌───────────────────── Traceback (most recent call last) ─────────────────────┐
│ e:\Repositories\graphrag-businessqa-evaluation\env\Lib\site-packages\graphr │
│ ag\cli\main.py:57 in _initialize_cli                                        │
│                                                                             │
│    54 │   ],                                                                │
│    55 ):                                                                    │
│    56 │   """Generate a default configuration file."""                      │
│ >  57 │   initialize_project_at(path=root)                                  │
│    58                                                                       │
│    59                                                                       │
│    60 @app.command("index")                                                 │
│                                                                             │
│ ┌──────────────────────────────── loca

In [ ]:
# indexing, only run once unless you want to update the index
# !graphrag index --root ./graphrag



⠋ GraphRAG Indexer 
Logging enabled at 
E:\Repositories\graphrag-businessqa-evaluation\graphrag\logs\indexing-engine.lo
g
⠋ GraphRAG Indexer 
⠋ GraphRAG Indexer 
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
⠋ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
🚀 create_base_text_units
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
Empty DataFrame
Columns: []
Index: []
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input 

[2024-11-14T01:38:05Z WARN  lance::dataset] No existing dataset at E:\Repositories\graphrag-businessqa-evaluation\graphrag\output\lancedb\default-entity-description.lance, it will be created
[2024-11-14T01:38:07Z WARN  lance::dataset] No existing dataset at E:\Repositories\graphrag-businessqa-evaluation\graphrag\output\lancedb\default-community-full_content.lance, it will be created
[2024-11-14T01:38:10Z WARN  lance::dataset] No existing dataset at E:\Repositories\graphrag-businessqa-evaluation\graphrag\output\lancedb\default-text_unit-text.lance, it will be created


## Query Engine

### Global Search

follows the implementations guide by the docs https://microsoft.github.io/graphrag/examples_notebooks/global_search/

In [ ]:
from graphrag_utils import setup_graphrag
import asyncio
import pandas as pd

api_key = os.environ["GRAPHRAG_API_KEY"]

with open('synthetic_data_big_context.json', 'r') as file:
    data = json.load(file)
queries = [response['question'] for response in data['responses']]
references = [response['ground_truth'] for response in data['responses']]

search_engine_global = setup_graphrag(api_key, model_name, community_level=2)

async def perform_search(query):
    result = await search_engine_global.asearch(query)
    return result.response

tasks = [perform_search(query) for query in queries]
results = await asyncio.gather(*tasks)

evaluation_samples = []
for query, result, reference in zip(queries, results, references):
    sample = SingleTurnSample(
        user_input=query,
        response=result,
        reference=reference,
        retrieved_contexts=[] 
    )
    evaluation_samples.append(sample)

evaluation_dataset = EvaluationDataset(samples=evaluation_samples)

In [104]:
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model=model_name))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

metrics = [
    FactualCorrectness(llm=evaluator_llm, mode="precision", name="FactualCorrectness_Precision"),
    FactualCorrectness(llm=evaluator_llm, mode="recall", name="FactualCorrectness_Recall"),
    Faithfulness(llm=evaluator_llm), 
]
results = evaluate(dataset=evaluation_dataset, metrics=metrics)
df = results.to_pandas()



Evaluating:  74%|███████▍  | 31/42 [02:53<00:52,  4.80s/it]Exception raised in Job[10]: TimeoutError()
Exception raised in Job[4]: TimeoutError()
Exception raised in Job[13]: TimeoutError()
Evaluating: 100%|██████████| 42/42 [04:07<00:00,  5.90s/it]


In [105]:
#pd.set_option('display.max_colwidth', 300) # default is 50 / None for unlimited
df

,user_input,retrieved_contexts,response,reference,FactualCorrectness_Precision,FactualCorrectness_Recall,faithfulness
0,How can SAP leverage its extensive data assets and AI capabilities to enhance customer experience while simultaneously driving operational efficiency across its product portfolio?,"[SAP is uniquely positioned to help our customers benefit from artificial intelligence by delivering Business AI, which we define as:\n- **Relevant:** AI delivered, from day one, across all SAP applications that customers use to run their busines...","SAP can enhance customer experience and drive operational efficiency by infusing Business AI across its product portfolio, ensuring AI is relevant, reliable, and responsible. By leveraging extensive data assets from over 25,000 customer tenants, ...","SAP can leverage its extensive data assets by embedding AI capabilities into its customer relationship management (CRM) and enterprise resource planning (ERP) systems. By utilizing predictive analytics, SAP can provide personalized customer exper...",NaN,0.29,0.500000
1,What strategies can SAP implement to ensure that its transition to a cloud-first model not only meets customer demands but also enhances its sustainability initiatives?,"[- **Sustainability** is at the core of SAP's vision and presents a unique opportunity for us to help customers in pursuing sustainability-focused opportunities and innovation, preparing to mitigate climate, resource and social-related risks, and...","SAP can enhance its cloud-first model and sustainability initiatives by leveraging its comprehensive portfolio of sustainability solutions to integrate sustainability data into core ERP systems, optimizing customer performance. By focusing on con...","SAP can implement a multi-faceted strategy that includes optimizing its cloud infrastructure for energy efficiency, utilizing renewable energy sources, and developing sustainability-focused applications. By adopting a green cloud strategy, SAP ca...",0.30,NaN,0.800000
2,How can SAP utilize generative AI to enhance its product development processes while ensuring compliance with industry regulations?,"[SAP is at the forefront of this technological wave, uniquely combining generative AI with business data. We are committed to providing AI solutions that are integral to our customers' most critical business processes, ensuring they are built-in,...","SAP can enhance its product development processes by infusing generative AI across its product portfolio, leveraging unique business data to ensure relevant and reliable AI solutions. To maintain compliance with industry regulations, SAP is commi...","SAP can utilize generative AI to streamline product development by automating the generation of design specifications, testing scenarios, and compliance documentation. By integrating AI tools that can analyze regulatory requirements and generate ...",0.17,0.08,0.833333
3,In what ways can SAP's Business Technology Platform (BTP) be leveraged to foster collaboration across its diverse product lines while enhancing customer engagement?,[# Our Strategic Priorities\n## Product Strategy Deep Dive\n### Innovate and focus the portfolio\n\nWe have many strengths to build on.\n\nA diverse and robust portfolio of business applications is certainly one of these. We believe in the value ...,"SAP's Business Technology Platform (BTP) enhances collaboration across diverse product lines by providing a unified foundation for integration, enabling live data flows and cross-analytics that facilitate interconnected business processes. This i...","SAP's Business Technology Platform (BTP) can be leveraged to create a unified ecosystem that integrates various product lines, enabling seamless data sharing and collaboration. By utilizing BTP's capabilities for application development, integrat...",0.27,NaN,0.636364
4,How can SAP align its AI initiatives with its corporate sustainability goals to create a competitive advantage in the market?,[The 